In [1]:
from os.path import join, basename, abspath, exists
from os import listdir, makedirs

from keras import backend as K
from keras.models import load_model
from matplotlib import pyplot as plt

import pandas as pd
import numpy as np
import cv2
import time

import pickle
# for scaling and inverse_transform
from sklearn.preprocessing import MinMaxScaler

from jaad_data import JAAD

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
model_path = "conf5_30ffuture-model.h5"
model = load_model(model_path)
n_seq = 30
n_seq_future = 30

jaad_path = "/home/kara9147/jaad2"
annotation_path = join(jaad_path, 'annotations')

imdb = JAAD(data_path = jaad_path)

In [3]:
def play():
    print("play..")
    global batch
    img_width, img_height = 960, 540
    start_time_video = time.time()
    clip_path = "/home/kara9147/JAAD/JAAD_clips/" + vid + ".mp4"
    cap = cv2.VideoCapture(clip_path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print( length, frames[0] )
    
    # Time to read all frames, predict and put bounding boxes around them, and show them.
    i = 0
    total_time = 0.0
    pred = True

    # Capture frame-by-frame
    ret = True
    
    while(ret and i <  frames[0] + n_seq):
        ret, origimg = cap.read()
        i += 1
    print(i)
    i = 0
    while(ret and i < lstm_seq.shape[0] - (n_seq + n_seq_future) -1):
        ret, origimg = cap.read()
        batch = lstm_seq[i:i + n_seq]
        print(batch)
       
        i = i + 1
        #print("Processing {} th frame".format(i))
        if (ret != False ):
            #print(img.shape)
            img = cv2.resize(origimg, (img_width, img_height))
            current = time.time()
            ##################################PREDICTION######################
            y_hat = model.predict(np.expand_dims(batch, axis=0))
            print(y_hat)

            #batch =  np.delete(batch, 0, axis = 0)
            #batch =  np.append(batch, y_hat, axis = 0)
            #print(len(batch))
            
            end = time.time()
            diff = end - current
            total_time  = total_time  + diff
            #print(end - current)
            #print("Time spent for predicting: {0}".format(diff))

            # 4: Decode the raw prediction `y_pred`
            inv_yhat = scaler.inverse_transform(y_hat)[0]
            print(inv_yhat)
            #inv_yhat = y_hat[0]
           
            np.set_printoptions(precision=2, suppress=True, linewidth=90)
            # print("Predicted boxes:\n")
            # print('   class   conf xmin   ymin   xmax   ymax')

            #print(y_pred_decoded)

            #print(time.time() - start_time)

            if pred:
                xmin = int(inv_yhat[0] / 2)
                ymin = int(inv_yhat[1] / 2)
                xmax = int(inv_yhat[2] / 2)
                ymax = int(inv_yhat[3] / 2)
            else:
                xmin = int(bb_cross[ped_b[ped_index]][i][0] / 2)
                ymin = int(bb_cross[ped_b[ped_index]][i][1] / 2)
                xmax = int(bb_cross[ped_b[ped_index]][i][2] / 2)
                ymax = int(bb_cross[ped_b[ped_index]][i][3] / 2)

            print((xmin, ymin))
            print((xmax, ymax))
            # 1920x1080
            #cv2.rectangle(img, (0,  0 ), (1910, 1070), (255, 0, 0), 2)

            # Draw the predicted boxes in blue
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax ), (255, 0, 0), 1)

            #xmin_ = int (df[i:i+1][0] / 2)
            #ymin_ = int (df[i:i+1][1] / 2)
            #xmax_ = int (df[i:i+1][2] / 2)
            #ymax_ = int (df[i:i+1][3] / 2)
            
            xmin_ = int (df[i+n_seq -1 : i+n_seq][0] / 2)
            ymin_ = int (df[i+n_seq -1 : i+n_seq][1] / 2)
            xmax_ = int (df[i+n_seq -1 : i+n_seq][2] / 2)
            ymax_ = int (df[i+n_seq -1 : i+n_seq][3] / 2)
            
            # Draw the present boxes in green
            #cv2.rectangle(img, (xmin_, ymin_), (xmax_, ymax_ ), (0, 255, 0), 1)
            
            # Draw the actual future boxes in red
            cv2.rectangle(img, (int(df[i+(n_seq + n_seq_future) -1 : i + (n_seq + n_seq_future)][0] / 2),
                          int(df[i+(n_seq + n_seq_future) -1 : i+ (n_seq + n_seq_future)][1] / 2)),
                          (int(df[i+(n_seq + n_seq_future) -1 : i+ (n_seq + n_seq_future)][2] / 2),
                          int(df[i+(n_seq + n_seq_future) -1 : i+ (n_seq + n_seq_future)][3] / 2))
                          , (0, 0,255), 1)


            # Display the resulting frame
            cv2.imshow('frame',img)

            # waitKey: 0, wait indefinitely
            if cv2.waitKey(33) & 0xFF == ord('q'):
                break

    end_time_video = time.time()
    print("No of frames: {}".format(i))
    print("Total Time: {}".format(total_time))
    print("fps: {}".format(i / (total_time)))

    # When everything done, release the capture
    cap.release()
    cv2.destroyAllWindows()


In [12]:
#342,341,  
vid = "video_0338"
ped_index = 0
anno = imdb._get_annotations(vid)
bb_cross={}
ped_b = [k for k,v in anno["ped_annotations"].items() if k.endswith('b')]
print(len(ped_b))
for i, p in enumerate(ped_b):
    #if (1 in anno["ped_annotations"][p]["behavior"]["cross"]):
        bb_cross[p] = (anno["ped_annotations"][p]["bbox"])
        #print(p)
        #print(i)
        
#print(ped_b)
#for k,v in bb_cross.items():
#    print(k)

print(ped_b[ped_index])
lstm_seq =  bb_cross[ped_b[ped_index]]
frames = (anno['ped_annotations'][ped_b[ped_index]]['frames'][0], len(anno['ped_annotations'][ped_b[ped_index]]['frames']))
print(frames)
df = pd.DataFrame(lstm_seq)
df = df.astype('float32')
scaler = MinMaxScaler(feature_range=(0, 1))
lstm_seq = scaler.fit_transform(lstm_seq)

play()

2
0_338_2681b
(0, 210)
play..
210 0
30
[[0.19 0.65 0.13 0.  ]
 [0.19 0.65 0.13 0.01]
 [0.19 0.65 0.12 0.01]
 [0.19 0.63 0.12 0.01]
 [0.18 0.63 0.12 0.01]
 [0.18 0.63 0.12 0.01]
 [0.18 0.63 0.12 0.01]
 [0.18 0.63 0.11 0.01]
 [0.18 0.63 0.11 0.01]
 [0.18 0.6  0.11 0.03]
 [0.17 0.58 0.11 0.04]
 [0.17 0.56 0.11 0.06]
 [0.17 0.53 0.11 0.07]
 [0.17 0.56 0.11 0.09]
 [0.17 0.6  0.11 0.11]
 [0.16 0.63 0.1  0.13]
 [0.16 0.65 0.1  0.14]
 [0.16 0.72 0.1  0.16]
 [0.15 0.79 0.09 0.17]
 [0.15 0.84 0.09 0.18]
 [0.15 0.91 0.09 0.2 ]
 [0.14 0.98 0.08 0.22]
 [0.14 0.91 0.08 0.2 ]
 [0.14 0.81 0.08 0.19]
 [0.14 0.74 0.08 0.17]
 [0.13 0.65 0.07 0.15]
 [0.13 0.58 0.07 0.14]
 [0.13 0.51 0.07 0.13]
 [0.12 0.47 0.06 0.13]
 [0.12 0.4  0.06 0.12]]
[[0.03 0.31 0.02 0.39]]
[245.67 625.46 324.77 830.56]
(122, 312)
(162, 415)
[[0.19 0.65 0.13 0.01]
 [0.19 0.65 0.12 0.01]
 [0.19 0.63 0.12 0.01]
 [0.18 0.63 0.12 0.01]
 [0.18 0.63 0.12 0.01]
 [0.18 0.63 0.12 0.01]
 [0.18 0.63 0.11 0.01]
 [0.18 0.63 0.11 0.01]
 [0.18 0.6

[[0.02 0.55 0.01 0.55]]
[241.44 635.68 322.21 858.63]
(120, 317)
(161, 429)
[[0.17 0.56 0.11 0.06]
 [0.17 0.53 0.11 0.07]
 [0.17 0.56 0.11 0.09]
 [0.17 0.6  0.11 0.11]
 [0.16 0.63 0.1  0.13]
 [0.16 0.65 0.1  0.14]
 [0.16 0.72 0.1  0.16]
 [0.15 0.79 0.09 0.17]
 [0.15 0.84 0.09 0.18]
 [0.15 0.91 0.09 0.2 ]
 [0.14 0.98 0.08 0.22]
 [0.14 0.91 0.08 0.2 ]
 [0.14 0.81 0.08 0.19]
 [0.14 0.74 0.08 0.17]
 [0.13 0.65 0.07 0.15]
 [0.13 0.58 0.07 0.14]
 [0.13 0.51 0.07 0.13]
 [0.12 0.47 0.06 0.13]
 [0.12 0.4  0.06 0.12]
 [0.11 0.4  0.06 0.13]
 [0.11 0.4  0.06 0.13]
 [0.11 0.37 0.05 0.13]
 [0.1  0.37 0.05 0.13]
 [0.1  0.37 0.05 0.13]
 [0.1  0.37 0.05 0.14]
 [0.09 0.4  0.04 0.17]
 [0.09 0.4  0.04 0.18]
 [0.08 0.4  0.04 0.19]
 [0.08 0.42 0.04 0.2 ]
 [0.08 0.42 0.04 0.2 ]]
[[0.02 0.59 0.01 0.59]]
[238.55 637.44 320.78 864.28]
(119, 318)
(160, 432)
[[0.17 0.53 0.11 0.07]
 [0.17 0.56 0.11 0.09]
 [0.17 0.6  0.11 0.11]
 [0.16 0.63 0.1  0.13]
 [0.16 0.65 0.1  0.14]
 [0.16 0.72 0.1  0.16]
 [0.15 0.79 0.09 0.

[[0.02 0.63 0.03 0.72]]
[241.67 638.89 332.82 885.94]
(120, 319)
(166, 442)
[[0.14 0.81 0.08 0.19]
 [0.14 0.74 0.08 0.17]
 [0.13 0.65 0.07 0.15]
 [0.13 0.58 0.07 0.14]
 [0.13 0.51 0.07 0.13]
 [0.12 0.47 0.06 0.13]
 [0.12 0.4  0.06 0.12]
 [0.11 0.4  0.06 0.13]
 [0.11 0.4  0.06 0.13]
 [0.11 0.37 0.05 0.13]
 [0.1  0.37 0.05 0.13]
 [0.1  0.37 0.05 0.13]
 [0.1  0.37 0.05 0.14]
 [0.09 0.4  0.04 0.17]
 [0.09 0.4  0.04 0.18]
 [0.08 0.4  0.04 0.19]
 [0.08 0.42 0.04 0.2 ]
 [0.08 0.42 0.04 0.2 ]
 [0.08 0.42 0.03 0.21]
 [0.07 0.33 0.03 0.19]
 [0.07 0.23 0.03 0.18]
 [0.07 0.12 0.03 0.16]
 [0.06 0.02 0.02 0.14]
 [0.06 0.02 0.02 0.15]
 [0.06 0.02 0.02 0.17]
 [0.05 0.   0.01 0.17]
 [0.05 0.   0.01 0.19]
 [0.04 0.09 0.01 0.22]
 [0.04 0.21 0.01 0.26]
 [0.04 0.3  0.01 0.3 ]]
[[0.01 0.64 0.01 0.74]]
[234.02 639.71 320.2  890.32]
(117, 319)
(160, 445)
[[0.14 0.74 0.08 0.17]
 [0.13 0.65 0.07 0.15]
 [0.13 0.58 0.07 0.14]
 [0.13 0.51 0.07 0.13]
 [0.12 0.47 0.06 0.13]
 [0.12 0.4  0.06 0.12]
 [0.11 0.4  0.06 0.

[[0.12 0.38 0.13 0.59]]
[315.47 628.33 420.7  863.84]
(157, 314)
(210, 431)
[[0.1  0.37 0.05 0.14]
 [0.09 0.4  0.04 0.17]
 [0.09 0.4  0.04 0.18]
 [0.08 0.4  0.04 0.19]
 [0.08 0.42 0.04 0.2 ]
 [0.08 0.42 0.04 0.2 ]
 [0.08 0.42 0.03 0.21]
 [0.07 0.33 0.03 0.19]
 [0.07 0.23 0.03 0.18]
 [0.07 0.12 0.03 0.16]
 [0.06 0.02 0.02 0.14]
 [0.06 0.02 0.02 0.15]
 [0.06 0.02 0.02 0.17]
 [0.05 0.   0.01 0.17]
 [0.05 0.   0.01 0.19]
 [0.04 0.09 0.01 0.22]
 [0.04 0.21 0.01 0.26]
 [0.04 0.3  0.01 0.3 ]
 [0.04 0.42 0.01 0.34]
 [0.04 0.51 0.01 0.38]
 [0.03 0.49 0.01 0.37]
 [0.03 0.49 0.01 0.37]
 [0.02 0.47 0.01 0.35]
 [0.02 0.47 0.   0.35]
 [0.01 0.44 0.   0.34]
 [0.01 0.42 0.   0.35]
 [0.01 0.37 0.   0.35]
 [0.   0.35 0.   0.36]
 [0.   0.3  0.   0.36]
 [0.   0.33 0.   0.37]]
[[0.13 0.39 0.15 0.62]]
[325.59 628.7  439.51 869.2 ]
(162, 314)
(219, 434)
[[0.09 0.4  0.04 0.17]
 [0.09 0.4  0.04 0.18]
 [0.08 0.4  0.04 0.19]
 [0.08 0.42 0.04 0.2 ]
 [0.08 0.42 0.04 0.2 ]
 [0.08 0.42 0.03 0.21]
 [0.07 0.33 0.03 0.

[[0.14 0.43 0.16 0.62]]
[335.83 630.32 450.19 870.07]
(167, 315)
(225, 435)
[[0.06 0.02 0.02 0.17]
 [0.05 0.   0.01 0.17]
 [0.05 0.   0.01 0.19]
 [0.04 0.09 0.01 0.22]
 [0.04 0.21 0.01 0.26]
 [0.04 0.3  0.01 0.3 ]
 [0.04 0.42 0.01 0.34]
 [0.04 0.51 0.01 0.38]
 [0.03 0.49 0.01 0.37]
 [0.03 0.49 0.01 0.37]
 [0.02 0.47 0.01 0.35]
 [0.02 0.47 0.   0.35]
 [0.01 0.44 0.   0.34]
 [0.01 0.42 0.   0.35]
 [0.01 0.37 0.   0.35]
 [0.   0.35 0.   0.36]
 [0.   0.3  0.   0.36]
 [0.   0.33 0.   0.37]
 [0.01 0.37 0.   0.38]
 [0.01 0.4  0.   0.39]
 [0.02 0.44 0.   0.41]
 [0.02 0.47 0.   0.41]
 [0.02 0.49 0.   0.43]
 [0.02 0.51 0.01 0.44]
 [0.03 0.53 0.01 0.46]
 [0.03 0.56 0.01 0.47]
 [0.03 0.6  0.02 0.48]
 [0.02 0.63 0.03 0.49]
 [0.02 0.67 0.04 0.5 ]
 [0.02 0.7  0.04 0.51]]
[[0.16 0.44 0.18 0.65]]
[346.96 631.12 459.11 874.08]
(173, 315)
(229, 437)
[[0.05 0.   0.01 0.17]
 [0.05 0.   0.01 0.19]
 [0.04 0.09 0.01 0.22]
 [0.04 0.21 0.01 0.26]
 [0.04 0.3  0.01 0.3 ]
 [0.04 0.42 0.01 0.34]
 [0.04 0.51 0.01 0.

...

[[0.48 0.51 0.52 0.72]]
[608.12 633.91 749.34 886.71]
(304, 316)
(374, 443)
[[0.14 0.72 0.12 0.6 ]
 [0.15 0.72 0.13 0.6 ]
 [0.15 0.7  0.14 0.62]
 [0.16 0.67 0.15 0.63]
 [0.16 0.65 0.17 0.63]
 [0.16 0.63 0.18 0.65]
 [0.16 0.7  0.19 0.65]
 [0.16 0.74 0.19 0.65]
 [0.16 0.81 0.2  0.65]
 [0.17 0.74 0.2  0.66]
 [0.18 0.7  0.2  0.67]
 [0.19 0.63 0.2  0.67]
 [0.2  0.56 0.2  0.68]
 [0.21 0.58 0.2  0.68]
 [0.22 0.63 0.2  0.69]
 [0.23 0.65 0.2  0.69]
 [0.24 0.67 0.22 0.69]
 [0.25 0.67 0.24 0.69]
 [0.25 0.7  0.26 0.69]
 [0.26 0.7  0.27 0.68]
 [0.26 0.67 0.27 0.68]
 [0.26 0.67 0.27 0.69]
 [0.26 0.65 0.27 0.69]
 [0.26 0.63 0.27 0.69]
 [0.27 0.65 0.28 0.69]
 [0.28 0.67 0.28 0.7 ]
 [0.29 0.7  0.29 0.7 ]
 [0.31 0.67 0.3  0.69]
 [0.32 0.67 0.31 0.69]
 [0.34 0.65 0.31 0.68]]
[[0.49 0.51 0.53 0.71]]
[615.74 633.98 755.72 885.11]
(307, 316)
(377, 442)
[[0.15 0.72 0.13 0.6 ]
 [0.15 0.7  0.14 0.62]
 [0.16 0.67 0.15 0.63]
 [0.16 0.65 0.17 0.63]
 [0.16 0.63 0.18 0.65]
 [0.16 0.7  0.19 0.65]
 [0.16 0.74 0.19 0.

[[0.62 0.75 0.67 0.78]]
[722.95 644.44 879.46 896.22]
(361, 322)
(439, 448)
[[0.2  0.56 0.2  0.68]
 [0.21 0.58 0.2  0.68]
 [0.22 0.63 0.2  0.69]
 [0.23 0.65 0.2  0.69]
 [0.24 0.67 0.22 0.69]
 [0.25 0.67 0.24 0.69]
 [0.25 0.7  0.26 0.69]
 [0.26 0.7  0.27 0.68]
 [0.26 0.67 0.27 0.68]
 [0.26 0.67 0.27 0.69]
 [0.26 0.65 0.27 0.69]
 [0.26 0.63 0.27 0.69]
 [0.27 0.65 0.28 0.69]
 [0.28 0.67 0.28 0.7 ]
 [0.29 0.7  0.29 0.7 ]
 [0.31 0.67 0.3  0.69]
 [0.32 0.67 0.31 0.69]
 [0.34 0.65 0.31 0.68]
 [0.35 0.63 0.32 0.68]
 [0.35 0.65 0.34 0.69]
 [0.36 0.67 0.36 0.7 ]
 [0.36 0.7  0.38 0.71]
 [0.36 0.72 0.38 0.71]
 [0.37 0.74 0.39 0.72]
 [0.37 0.77 0.39 0.72]
 [0.37 0.79 0.4  0.72]
 [0.38 0.72 0.4  0.71]
 [0.39 0.65 0.39 0.71]
 [0.4  0.58 0.39 0.7 ]
 [0.41 0.58 0.4  0.71]]
[[0.64 0.77 0.69 0.8 ]]
[736.83 645.29 892.03 899.1 ]
(368, 322)
(446, 449)
[[0.21 0.58 0.2  0.68]
 [0.22 0.63 0.2  0.69]
 [0.23 0.65 0.2  0.69]
 [0.24 0.67 0.22 0.69]
 [0.25 0.67 0.24 0.69]
 [0.25 0.7  0.26 0.69]
 [0.26 0.7  0.27 0.

[[0.76 0.78 0.79 0.88]]
[837.26 645.42 979.36 913.62]
(418, 322)
(489, 456)
[[0.27 0.65 0.28 0.69]
 [0.28 0.67 0.28 0.7 ]
 [0.29 0.7  0.29 0.7 ]
 [0.31 0.67 0.3  0.69]
 [0.32 0.67 0.31 0.69]
 [0.34 0.65 0.31 0.68]
 [0.35 0.63 0.32 0.68]
 [0.35 0.65 0.34 0.69]
 [0.36 0.67 0.36 0.7 ]
 [0.36 0.7  0.38 0.71]
 [0.36 0.72 0.38 0.71]
 [0.37 0.74 0.39 0.72]
 [0.37 0.77 0.39 0.72]
 [0.37 0.79 0.4  0.72]
 [0.38 0.72 0.4  0.71]
 [0.39 0.65 0.39 0.71]
 [0.4  0.58 0.39 0.7 ]
 [0.41 0.58 0.4  0.71]
 [0.43 0.58 0.41 0.71]
 [0.44 0.56 0.42 0.72]
 [0.45 0.56 0.43 0.72]
 [0.46 0.56 0.43 0.73]
 [0.47 0.63 0.45 0.72]
 [0.47 0.7  0.46 0.72]
 [0.47 0.74 0.48 0.71]
 [0.48 0.81 0.49 0.7 ]
 [0.48 0.79 0.49 0.71]
 [0.48 0.79 0.5  0.72]
 [0.48 0.77 0.5  0.72]
 [0.48 0.74 0.5  0.73]]
[[0.8  0.74 0.82 0.89]]
[ 865.05  643.88 1002.03  915.4 ]
(432, 321)
(501, 457)
[[0.28 0.67 0.28 0.7 ]
 [0.29 0.7  0.29 0.7 ]
 [0.31 0.67 0.3  0.69]
 [0.32 0.67 0.31 0.69]
 [0.34 0.65 0.31 0.68]
 [0.35 0.63 0.32 0.68]
 [0.35 0.65 0.3

[[0.87 0.42 0.86 0.91]]
[ 921.28  630.05 1042.01  917.25]
(460, 315)
(521, 458)
[[0.37 0.77 0.39 0.72]
 [0.37 0.79 0.4  0.72]
 [0.38 0.72 0.4  0.71]
 [0.39 0.65 0.39 0.71]
 [0.4  0.58 0.39 0.7 ]
 [0.41 0.58 0.4  0.71]
 [0.43 0.58 0.41 0.71]
 [0.44 0.56 0.42 0.72]
 [0.45 0.56 0.43 0.72]
 [0.46 0.56 0.43 0.73]
 [0.47 0.63 0.45 0.72]
 [0.47 0.7  0.46 0.72]
 [0.47 0.74 0.48 0.71]
 [0.48 0.81 0.49 0.7 ]
 [0.48 0.79 0.49 0.71]
 [0.48 0.79 0.5  0.72]
 [0.48 0.77 0.5  0.72]
 [0.48 0.74 0.5  0.73]
 [0.5  0.74 0.51 0.72]
 [0.52 0.72 0.51 0.72]
 [0.54 0.72 0.51 0.71]
 [0.55 0.7  0.51 0.7 ]
 [0.56 0.65 0.52 0.71]
 [0.57 0.6  0.54 0.71]
 [0.57 0.53 0.55 0.71]
 [0.58 0.49 0.57 0.72]
 [0.58 0.56 0.58 0.72]
 [0.58 0.63 0.59 0.73]
 [0.58 0.67 0.6  0.74]
 [0.59 0.74 0.61 0.74]]
[[0.86 0.51 0.86 0.91]]
[ 913.79  633.88 1038.33  918.34]
(456, 316)
(519, 459)
[[0.37 0.79 0.4  0.72]
 [0.38 0.72 0.4  0.71]
 [0.39 0.65 0.39 0.71]
 [0.4  0.58 0.39 0.7 ]
 [0.41 0.58 0.4  0.71]
 [0.43 0.58 0.41 0.71]
 [0.44 0.56

[[0.92 0.72 0.93 0.93]]
[ 964.93  643.   1101.67  920.69]
(482, 321)
(550, 460)
[[0.47 0.74 0.48 0.71]
 [0.48 0.81 0.49 0.7 ]
 [0.48 0.79 0.49 0.71]
 [0.48 0.79 0.5  0.72]
 [0.48 0.77 0.5  0.72]
 [0.48 0.74 0.5  0.73]
 [0.5  0.74 0.51 0.72]
 [0.52 0.72 0.51 0.72]
 [0.54 0.72 0.51 0.71]
 [0.55 0.7  0.51 0.7 ]
 [0.56 0.65 0.52 0.71]
 [0.57 0.6  0.54 0.71]
 [0.57 0.53 0.55 0.71]
 [0.58 0.49 0.57 0.72]
 [0.58 0.56 0.58 0.72]
 [0.58 0.63 0.59 0.73]
 [0.58 0.67 0.6  0.74]
 [0.59 0.74 0.61 0.74]
 [0.59 0.81 0.62 0.75]
 [0.6  0.74 0.62 0.75]
 [0.6  0.7  0.62 0.75]
 [0.61 0.63 0.62 0.75]
 [0.63 0.67 0.62 0.77]
 [0.65 0.7  0.62 0.77]
 [0.67 0.74 0.62 0.78]
 [0.68 0.74 0.63 0.78]
 [0.69 0.77 0.64 0.78]
 [0.69 0.77 0.65 0.78]
 [0.7  0.77 0.67 0.78]
 [0.7  0.74 0.69 0.78]]
[[0.95 0.67 0.94 0.94]]
[ 983.1   640.81 1107.85  923.21]
(491, 320)
(553, 461)
[[0.48 0.81 0.49 0.7 ]
 [0.48 0.79 0.49 0.71]
 [0.48 0.79 0.5  0.72]
 [0.48 0.77 0.5  0.72]
 [0.48 0.74 0.5  0.73]
 [0.5  0.74 0.51 0.72]
 [0.52 0.72